In [ ]:
# Imports

!pip install marker-pdf pymupdf opencv-python pandas numpy matplotlib seaborn
!pip install textdistance nltk scikit-learn pillow
!pip install google-generativeai  
!pip install requests  

In [3]:
import os
import time
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
import fitz  # PyMuPDF
from PIL import Image
import textdistance
import nltk
from sklearn.metrics import accuracy_score
import re
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

print("✅ Setup complete!")

ModuleNotFoundError: No module named 'pandas'